<a href="https://colab.research.google.com/github/subhra004/End-to-end-Medical-Chatbot-genAI/blob/main/Parameter_Optimization_Assistant_using_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
import re

# Load the dataset
file_path = '/content/3dprinter.csv'
data = pd.read_csv(file_path)

# Preprocess the data
features = ['Roughness', 'Tension_strenght', 'Elongation']
parameters = ['layer_height', 'wall_thickness', 'infill_density', 'infill_pattern',
              'nozzle_temperature', 'Bed_temperature', 'Print_speed', 'Material', 'Fan_speed']

# Encode categorical variables
data_encoded = pd.get_dummies(data, columns=['infill_pattern', 'Material'])

# Scale the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(data_encoded[features])

# Train Nearest Neighbors Model
model = NearestNeighbors(n_neighbors=3, metric='euclidean')
model.fit(scaled_features)

# Function to process text input
def parse_input(user_input):
    desired = {'Roughness': None, 'Tension_strenght': None, 'Elongation': None}

    # Extract numerical values
    roughness = re.search(r'(low|medium|high) roughness', user_input, re.IGNORECASE)
    strength = re.search(r'(low|medium|high) tensile strength', user_input, re.IGNORECASE)
    elongation = re.search(r'(low|medium|high) elongation', user_input, re.IGNORECASE)

    # Map descriptions to numerical values
    mapping = {'low': 0.25, 'medium': 0.5, 'high': 0.75}

    if roughness:
        desired['Roughness'] = mapping[roughness.group(1).lower()]
    if strength:
        desired['Tension_strenght'] = mapping[strength.group(1).lower()]
    if elongation:
        desired['Elongation'] = mapping[elongation.group(1).lower()]

    return desired

# Function to get recommendations
def recommend_parameters(user_input):
    desired = parse_input(user_input)

    # Create input vector
    input_vector = np.array([[
        desired['Roughness'] if desired['Roughness'] is not None else data['Roughness'].mean(),
        desired['Tension_strenght'] if desired['Tension_strenght'] is not None else data['Tension_strenght'].mean(),
        desired['Elongation'] if desired['Elongation'] is not None else data['Elongation'].mean()
    ]])

    # Scale input vector
    scaled_input = scaler.transform(input_vector)

    # Find nearest neighbors
    distances, indices = model.kneighbors(scaled_input)
    recommendations = data.iloc[indices[0]][parameters]

    return recommendations

# Main chatbot loop
def chatbot():
    print("3D Printing Optimization Assistant: Type 'exit' to quit.")
    while True:
        user_input = input("Enter your requirements (e.g., 'I want high tensile strength and low roughness'): ")
        if user_input.lower() == 'exit':
            break
        recommendations = recommend_parameters(user_input)
        print("Recommended Parameters:")
        print(recommendations)

# Run chatbot
if __name__ == "__main__":
    chatbot()


3D Printing Optimization Assistant: Type 'exit' to quit.


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Recommended Parameters:
    layer_height  wall_thickness  infill_density infill_pattern  \
54          0.02             6.0              12           grid   
2           0.02             1.0              80           grid   
6           0.02             8.0              90           grid   

    nozzle_temperature  Bed_temperature  Print_speed Material  Fan_speed  
54                 205               65           40      pla         28  
2                  230               70           40      abs         50  
6                  250              100           40      abs        100  
